In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.12.0


In [3]:
from keras.datasets import fashion_mnist

In [4]:
fashion_mnist=fashion_mnist.load_data()

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist

In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)

In [9]:
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [11]:
!pip install keras-tuner

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-12-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [13]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [15]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        38432     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 128)               1982592   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 2,022,794
Trainable params: 2,022,794
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 13s 5ms/step - loss: 0.1402 - accuracy: 0.9470 - val_loss: 0.2483 - val_accuracy: 0.9168
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0989 - accuracy: 0.9623 - val_loss: 0.2859 - val_accuracy: 0.9163
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0687 - accuracy: 0.9744 - val_loss: 0.3053 - val_accuracy: 0.9180
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0498 - accuracy: 0.9822 - val_loss: 0.3745 - val_accuracy: 0.9153
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0369 - accuracy: 0.9866 - val_loss: 0.4495 - val_accuracy: 0.9115
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0301 - accuracy: 0.9889 - val_loss: 0.4649 - val_accuracy: 0.9118
Epoch 10/10
 542/1688 [========>.....................] - ETA: 6s - loss: 0.0184 - accuracy: 0.9933